In [ ]:
from google.colab import userdata
import requests
import pandas as pd

### Finding the hospital count by simply querying google maps

In [ ]:
county_data = pd.read_csv('/content/us-county-boundaries.csv', delimiter=';')

In [ ]:
def get_hospitals_in_county(county_name, api_key):
    url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query=hospitals+in+{county_name}+county&key={api_key}"
    response = requests.get(url)
    results = response.json().get('results', [])
    return len(results)

In [ ]:
api_key = userdata.get('googlemaps')
county_data['hospital_count'] = county_data['NAME'].apply(lambda x: get_hospitals_in_county(x, api_key))

In [ ]:
county_data[['NAMELSAD', 'hospital_count']].to_csv('hospitals_per_county.csv', index=False)

### Something new

In [ ]:
county_data = pd.read_csv('/content/us-county-boundaries.csv', delimiter=';')

In [ ]:
def get_hospitals_in_county(county_name, api_key):
    url = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query=hospitals+in+{county_name}+county&key={api_key}"
    response = requests.get(url)
    results = response.json().get('results', [])
    return results

In [ ]:
def get_transit_options(hospital_location, api_key):
    url = f"https://maps.googleapis.com/maps/api/directions/json?origin={hospital_location}&destination={hospital_location}&mode=transit&key={api_key}"
    response = requests.get(url)
    routes = response.json().get('routes', [])
    return routes


In [ ]:
api_key = userdata.get('googlemaps')
transit_data = []

for county in county_data['NAME']:
    hospitals = get_hospitals_in_county(county, api_key)
    for hospital in hospitals:
        hospital_location = f"{hospital['geometry']['location']['lat']},{hospital['geometry']['location']['lng']}"
        transit_options = get_transit_options(hospital_location, api_key)
        transit_data.append({
            'county': county,
            'hospital_name': hospital['name'],
            'transit_options': transit_options
        })

transit_df = pd.DataFrame(transit_data)


In [ ]:
transit_df.to_csv('transit_options_around_hospitals.csv', index=False)